Dexter Goodkind
# Mathematical Billiards


## Abstract

This project will aim to investigate the game of mathematical billiards, in which a billiard ball is treated as a point particle, travelling on a frictionless 'table' with no dissipation. The entire trajectory of the ball can, in principle, be determined solely by its initial position and its initial velocity (or simply initial direction) within a given set of boundaries for the table. The phase spaces formed by the trajectories will be examined, and concepts such as chaos and ergodicity will be explored in the context of billiard trajectories. 

## Background and general approach

A table is characterised by its boundaries. Within this project, only horizontal, vertical, circular (semi-circular) and elliptical boundaries will be considered. All reflections are characterised by the same simple rule: angle of incidence equals angle of reflection.

The physical problem occurs in two dimensions, so the approach taken will be to plot each trajectory on the xy-plane. 

There are two main ways in which one could go about generating trajectories. The first would be to initialise the ball and step it forward in time by some small step, measuring when it collides with the boundary. The other approach would be to view the trajectory as a series of line segments, each one dependent on the last. The latter approach is the one that this project chooses to adopt, as it uses mathematical analysis to determine accurate plots rather than simply using brute force. Generating equally precise plots with the former method will likely be much less computationally efficient. Furthermore, functionality for generating intermediate points (points between where the ball intersects the boundary), spaced according to the user's choice, is provided in the code below. This means that there is little disadvantage in terms of possible use of these points for analysis or generating animations.

The following parameterisation is used to characterise the next line segment the ball will travel along until it hits another boundary:

$$ \begin{pmatrix} x \\ y \end{pmatrix} = \begin{pmatrix} x_{init} \\ y_{init} \end{pmatrix} + \begin{pmatrix} v_0 \\ v_1 \end{pmatrix}t $$

where  $x_{init}$ and $y_{init}$ correspond to the coordinates of a known initial point; $v_0$ and $v_1$ are the components of the already calculated direction vector $\textbf{v}$ for this line segment and $t$ is some parameter. The code will solve for t on each piecewise part of the boundary, according to the specific equation that characterises that part of the boundary, and reflect the ball off of the boundary for which the t solution is the smallest. The parameter $t$ corresponds to some measure of time - negative $t$ corresponds to the ball travelling backwards in time along the prescribed trajectory. As such, solutions which give negative (and zero) t will be discounted. 

$\textbf{v}$ can be physically interpreted as the ball's velocity along each particular line segment, which will be kept normalised throughout as there is no dissipation.

### Limitations of the code

Though the techniques used to generate intersection points are analytic, the computer is constrained to rounding errors, which may cause the trajectories to slightly deviate from what they should theoretically be. This effect may be accentuated over many collisions, especially within tables that produce chaotic behaviours. 

## Chaos

Perhaps the most interesting part of analysing the game of mathematical billiards is the close relation the problem has to chaos theory. The problem provides an intuitive framework for understanding some of the basics of chaos theory. 

Phase space will become an important tool when analysing chaotic plots. In the context of mathematical billiards, although it is possible, in principle, to reduce phase space to two dimensions, an intuitive understanding of the problem can be obtained by considering a three dimensional phase space. The coordinates of a point in phase space will be determined by the x and y position of the ball, as well as theta - defined as the angle the direction vector makes with the x-axis at that point. Since three-dimensional phase space is unclear when viewed on a two-dimensional screen, it may often be more useful to look at two dimensional projections of the phase space. The phase space of a trajectory will help to understand whether a table produces chaotic or regular (non-chaotic) behaviour$^1$.

Chaos can be understood as extreme sensitivity to initial conditions. In other words, changing the initial conditions of the trajectory, even by a very small amount, should quickly lead to diverging trajectories. Another feature of chaotic behaviour is non-periodicity. A chaotic billiard table may have possible trajectories that are periodic. However, such trajectories are repelling, meaning that however close a ball gets to a point in phase space containing this periodic trajectory, the ball in fact diverges from the periodic trajectory, rather than being attracted to it. 

The concept of ergodicity also becomes important when analysing trajectories and whether the are chaotic. In the context of a dynamical system, such as a chaotic billiard table, to say that the trajectory is ergodic means that the ball does not concentrate into particular areas of the phase space more than others$^2$. Many chaotic systems display ergodicity, including some of the billiard trajectories to be found below. 

As a general rule, it will be easy to tell that a trajectory is regular by whether it produces some visible pattern. Examining the phase space of the trajectories will give a clearer indication as to whether the dynamics of a specific billiard are chaotic or regular. 

## Applications of mathematical billiards in physics

Billiard tables appear as natural analogies to many problems in physics, especially in optics and acoustics. In the context of statistical physics, the Boltzmann gas of elastically colliding particles in a box can be easily reduced to a corresponding billiard problem. Understanding the concept of ergodicity is also extremely important in statistical physics.  

## Key modules, methods and variables

In [ ]:
'''
It is recommended that before running the code, the user selects 'Kernel', 'Restart and Clear Output'
and then runs the code cell by cell, so that each animation can be viewed in turn. Animations should be stopped 
(by clicking 'stop interaction') after the user has finished viewing them, as having too many animations 
running at once will likely use up large amounts of memory and lead to lagging. 
'''

# importing modules and using IPython Magic to better format notebook for animations etc:

# IPython magic - setting backend:
%matplotlib notebook
%matplotlib tk

# tkinter (tk) windows allow us to view for generating animation windows
# notebook means that plots and animations are displayed within the notebook - this may lead to lagging.
# if the user wants smoother animations generated outside the notebook, they should remove (comment out) the
# %matplotlib notebook line. 
# having both notebook and tk will produce the following warning: 'Cannot change to a different GUI toolkit: tk. 
# Using notebook instead. ' This warning can be ignored, as we're not trying to change the GUI, we simply want
# the tk windows to appear within the notebook.

# tk windows are useful as they allow animations to appear within the notebook? 
# they also allow the user to interact with the plots and allow for a more user friendly environment. 

# importing necessary modules:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D          

# the following two lines ensure the user doesn't get a warning if they open too many figures at once.
import matplotlib as mpl
mpl.rcParams['figure.max_open_warning'] = 0
# However, the user should note that having too many active figures at once could use up too 
# much memory and cause the code to run at slower speeds.

In [ ]:
def normalise(vector):
    '''
    Function to normalise an n-dimensional vector. This will be important when we generate animations
    and phase plots, as we require the distance between each point to be consistent (this will not
    be the case just before the ball hits the next boundary, but if ball speed is small enough, 
    almost all the animation points generated will be evenly spaced, and this will be a good enough
    approximation to generate reasonable looking animations, as well as to analyse roughly evenly spaced
    points)
    
    Parameters:
        vector : list
            un-normalised vector
    
    Returns:
        vector: list
            normalised vector corresponding to input
    '''
    normalisation_const = np.sqrt(sum([ele**2 for ele in vector]))
    for i in range(len(vector)):
        vector[i] /= normalisation_const
    return vector

# functions for animation:
def update_line(num, data, line):
    '''
    Updates line with each new point in data.
    '''
    line.set_data(data[..., :num])
    return line,
def update_two_lines(num, data1, data2, line1, line2):
    '''
    Updates two lines with each new point in respective data.
    We will only need to call this function when generating animations for two trajectories
    at once.
    '''
    line1.set_data(data1[..., :num])
    line2.set_data(data2[..., :num])
    return line1, line2,

# to convert a direction vector to its corresponding angle:
def find_theta(direction_vector):
    '''
    Function to find the angle corresponding to a given direction vector.
    
    Parameters:
        direction_vector : list containing two numbers
            the two numbers correspond to the x and y direction of the ball, respectively.
            direction_vector doesn't have to be normalised, as we just compare ratios of 
            the two elements.
             
    Returns:
        theta: number between -pi and +pi.
            angle measured from the x-axis to the direction vector.
    '''
    v0, v1 = direction_vector[0], direction_vector[1]
    
    # there will be different formulae for each of the four quadrants the direction vector can end in
    # when it starts at the origin
    if v0 >= 0 and v1 > 0:
        theta = np.arctan(v1 / v0)
    elif v0 <= 0 and v1 > 0:
        theta = np.pi + np.arctan(v1 / v0)
    elif v0 >= 0 and v1 < 0:
        theta = np.arctan(v1 / v0)
    elif v0 <= 0 and v1 < 0:
        theta = np.arctan(v1 / v0) - np.pi
    elif v1 == 0:
        theta = 0 if v0 > 0 else np.pi
    
    return theta

# key variables:

ball_speed = 0.01 # will determine how fast ball appears in animation
# will determine how fast the ball appears in animation.
# will also determine the number of animation points generated - larger ball_speed corresponds to
# fewer animation points. This will become more important when we generate recurrence plots
# using these animation points.
# Fewer animation points also means the code will use up less memory, as it will have to store less data.
# If ball_speed is large enough, no intermediate points (between intersection points) will be generated - 
# this variable allows for optimisation, as we set it to be large when we don't want to generate intermediate
# points.

small_step = 1e-10  
# will be used for reflections - small step will be step back to previous point. 
# Should be much much smaller than scale of table - sufficiently small so that the next point
# i.e. the reflection of the previous point will always be inside the table. This is explained further
# under the Boundary Class heading. 

# small_step and ball_speed can be changed throughout the program as appropriate (though there
# is no real reason to change small_step)


## Object oriented approach

An object oriented approach, taken both for clarity of code and usability, allows the user to generate their own tables and trajectories very easily. Two classes are defined - a Boundary class for characterising boundaries and a Table class, which will take in a set of boundaries and use them as a 'billiard table' to generate trajectories. 

The user is encouraged to use the defined classes to create their own boundaries and tables and investigate the corresponding trajectories (they may encounter some errors, but generally unless the generated plot looks completely counter-intuitive, it is likely to be accurate). The 'inner' parameter is provided for each boundary so that if the plot looks counter-intuitive, an easy fix may be to set this parameter to True. The user may also have to alter the x and y limits of the figure in order to display the whole table.

Since an object oriented approach is used, the bulk of the important code is written within the two classes, while the analysis of trajectories will take place later in the project.

### Boundary class 

The complete set of table boundaries will be a collection of individual boundaries, each with their own specific categories. Straight lines must either fall into the 'horizontal' or 'vertical' categories, whilst categories of 'circle', 'ellipse', 'semi-circle left' and 'semi-circle right' are also supported. 


#### Collisions

The collide() method  will change the direction vector inputted according to the type of boundary, and where on the boundary the ball hits. Collisions with horizontal and vertical boundaries are the easiest to understand, as they just correspond to multiplying the y and x components of the direction vector, respectively, by $-1$. Collisions with curved boundaries are more difficult to code for. The approach taken is to consider the collision as a reflection across the normal at the boundary. This is done by taking a small step back in time along the previous line segment (this small step is controlled by the small_step variable), and reflecting this point across the normal at the boundary. 

The equation of the normal for a circle centred at the origin at a point on the boundary, $ (x_{boundary}, y_{boundary}) $ is simply $ y = mx $ where the gradient, $m$, is given by $ m = y_{boundary}/x_{boundary} $. (This can be easily translated for circles that are not centred at the origin).

To find the equation of the normal for an ellipse centred at the origin at a point on the boundary, the parameterisation $ x = a\cos(p), y = p\sin(p) $ may be used, where $p$ is some parameter that varies between $0$ and $2\pi$, and $a$ and $b$ are the semi-major and semi-minor axes of the ellipse respectively (it is assumed $a \geq b$, with $a=b$ corresponding to a circle). The equation for the normal at a boundary point with a certain $p$ is then given by $ ax\sec(p) - by\csc(p) = a^2 - b^2 $. 

Once the gradients ($m$) and y-intercepts ($c$) of the normals are calculated, the following known transformation matrix for reflection along the line $y = mx + c$ is used (this is a known result in linear algebra):

$$ \begin{pmatrix} x' \\ y' \\ 1 \end{pmatrix} = \begin{pmatrix} \frac{1-m^2}{1+m^2} & \frac{2m}{1+m^2} & \frac{-2mc}{1+m^2} \\ \frac{2m}{1+m^2} & \frac{m^2-1}{1+m^2} & \frac{2c}{1+m^2} \\ 0 & 0 & 1 \end{pmatrix}\begin{pmatrix} x \\ y \\ 1 \end{pmatrix}. $$

The new point generated under this reflection is then used to calculate the new direction vector post-collision as the vector between this new point and the point on the boundary.


#### Finding t

In order to find the value of the parameter $t$ at which the ball hits the boundary, the find_t() class method is used. All negative values of t are to be ignored, so if negative values are obtained, t is set to infinity, which will be equivalent to the ball never hitting the boundary.

The parameterisation $x = x_{init} + v_0t$, $y = y_{init} +v_1t$ is used, and substituted into the equations for each part of the piecewise boundary. In the case of circles and ellipses, this produces a quadratic with two solutions, for which the smallest (positive, real) solution is taken which satisfies any extra conditions is taken (the self.inner attribute may become relevant here).

After finding $t$ for each boundary of the table, the ball is collided with the boundary for which $t$ is minimised. In other words, the ball collides with the first boundary it comes into contact with.


#### Other class methods

Functionality for plotting the boundaries and finding the boundary points is also included. The method for finding boundary points will be important when collisions are generated in the Table class.

In [ ]:
class Boundary:
    
    def __init__(self, category, xlims=None, ylims=None, inner=False, centre=None, radius=None, a=None, b=None):
        '''
        Initialisation function for boundary class. The only parameter required for every boundary is category,
        but it is assumed that the user will input all required parameters for the specific boundary they
        initialise. 
        
        The optional parameters can be defaulted to None, in case they are never used (with the exception
        of inner which is a boolean, initialised to False).
        
        
        Parameters:
            category : string
                Choice beteen 'horizontal', 'vertical', 'circle', 'ellipse', 'semi-circle left' and 'semi-circle right'.
                'horizontal' corresponds to horizontal lines. Requires xlims, ylims to be declared.
                'vertical' corresponds to vertical lines. Requires xlims, ylims to be inputted.
                'circle' corresponds to a full circumference. Requires centre and radius to be inputted.
                'ellipse' corresponds to a full elliptical boundary. Requires (centre), a and b to be inputted.
                    We deal only with ellipses whose semi-major axis is parallel to the x-axis.
                'semi-circle left' corresponds to a semi-circle which would lie entirely on the left-hand side of 
                    the y-axis if centred at the origin. Requires centre and radius to be inputted.
                'semi-circle right' corresponds to a semi-circle which would lie entirely on the right-hand side 
                    of the y-axis if centred at the origin. Requires centre and radius to be inputted.
                    
                The distinction between circles and semi-circles is mainly introduced for the purpose of plotting 
                the boundaries - the mathematics underlying them is very similar.

                    
            xlims (optional) : list
                The x limits within which the boundary is constrained. Can contain either one or two elements (one 
                if the boundary is vertical so that every point on the boundary has the same x value)
                
            ylims (optional) : list
                The y limits within which the boundary is constrained. Can contain either one or two elements (one 
                if the boundary is horizontal so that every point on the boundary has the same y value)
                
            inner (optional) : boolean
                True if the boundary lies within a shape (particularly applicable for semi-circles).
                The parameter allows for code to be cleaner and clearer, though it is up to the user to 
                decide based on the situation whether to set it to true. As a general rule, it should be set to 
                true if the boundary is contained within some outer boundary. If the plot generated looks 
                counter-intuitive, an easy fix may be to set this parameter to True.
                
            centre (optional) : tuple
                In the form (x_c, y_c) where x_c and y_c are the x and y coordinates of the centre (of the
                circle or ellipse) respectively.
                
            radius (optional) : number
                Radius of the circle or semi-circle
                
            a (optional) : number
                Semi-major axis of an ellipse
                
            b (optional) : number
                Semi-minor axis of an ellipse
                
        '''
        # initialising class attributes:
        self.category = category
        self.xlims = xlims
        self.ylims = ylims
        self.inner = inner
        self.centre = centre
        self.radius = radius
        self.a = a
        self.b = b
        # if we're dealing with an ellipse, we will want to find another constant c, which will correspond to 
        # the distance between the foci of the ellipse and the centre:
        if category == 'ellipse':
            self.c = np.sqrt(a**2 - b**2)
        
        # we can set self.t to be its actual value after we call the find_t() class method.
        self.t = None
        
    def plot(self, fig):
        '''
        Method for plotting boundaries. Generally, we will call this function within the Table class, unless
        we want to plot the boundaries without plotting the trajectories. 
        
        Parameters:
            fig : matplotlib Figure
                This is the figure on which we will plot the boundaries.
        '''
        if self.category == 'vertical':
            plt.plot([self.xlims[0], self.xlims[0]], self.ylims, color='red', figure=fig)
        elif self.category == 'horizontal':
            plt.plot(self.xlims, [self.ylims[0], self.ylims[0]], color='red', figure=fig)
        elif self.category == 'circle':
            circle = plt.Circle(self.centre, self.radius, color='red', fill=False, figure=fig)
            plt.gcf().gca().add_artist(circle)          #explain what this line does?
        elif self.category == 'ellipse':
            ellipse = patches.Ellipse(self.centre, width=2*self.a, height=2*self.b, color='red', fill=False, figure=fig)
            plt.gcf().gca().add_artist(ellipse)
            # for an ellipse, we want to plot the foci too, as these will play an important role 
            # when we analyse the trajectories. 
            # check that this works for ellipses in general.
            plt.plot([self.centre[0] + self.c], [self.centre[1]], marker='o', markersize=3, color='green', figure=fig)    #change 0 to self.centre[0] i think
            plt.plot([self.centre[0] - self.c], [self.centre[1]], marker='o', markersize=3, color='green', figure=fig)
        elif self.category == 'semi-circle left' or self.category == 'semi-circle right':
            # for semi-circles, we need to define the angles at which they start and end (in degrees).
            if self.category == 'semi-circle left':
                start, end = 90, 270
            elif self.category == 'semi-circle right':
                start, end = 270, 90
            
            semi_circle = patches.Arc(self.centre, width=2*self.radius, height=2*self.radius, 
                                          theta1=start, theta2=end, color='red', linewidth=1.5, figure=fig)          # thetas are in degrees
            plt.gcf().gca().add_artist(semi_circle)
            
    def find_t(self, x_init, y_init, v0, v1):
        '''
        Method for finding the value of the parameter t at which the ball next hits the boundary.
        Setting t to infinity will correspond physically to the ball never hitting the boundary.
        
        Parameters:
            x_init : number
            y_init : number
                x_init and y_init should be the x and y coordinates, respectively, of a previous point 
                on the line segment for which we're trying to find the next collision. When we code the Table class,
                (x_init, y_init) will either be taken as our initial point or the last point where the ball hit
                the boundary.
                
            v0 : number
            v1 : number
                These are the x and y components (respectively) of the normalised direction vector before collision.
                
        Returns:
            t : number
                The value of the parameter at which the ball hits the boundary. Will either be a positive finite
                number or infiniy.
        '''
        
        if self.category == 'vertical':
            # the following if/else statement is there to avoid error warnings that we may be dividing by zero.
            if v0 != 0:
                t = (self.xlims[0] - x_init) / v0
            else:
                t = np.inf
            
            # if the t doesn't correspond to a point on our physical boundary, we don't want the ball to ever
            # hit the boundary (i.e. if the y_boundary is outside ylims, we set t to infinity):  
            if t > 0 and t != np.inf:
                if y_init + v1 * t < self.ylims[0] or y_init + v1 * t > self.ylims[1]:
                    t = np.inf
                    
        elif self.category == 'horizontal':
            # the following if/else statement is there to avoid error warnings that we may be dividing by zero.
            if v1 != 0:
                t = (self.ylims[0] - y_init) / v1
            else:
                t = np.inf
            
            # if the t doesn't correspond to a point on our physical boundary, we don't want the ball to ever
            # hit the boundary (i.e. if the x_boundary is outside xlims, we set t to infinity):  
            if t > 0 and t != np.inf:
                if x_init + v0 * t < self.xlims[0] or x_init + v0 * t > self.xlims[1]:
                    t = np.inf
            
        elif (self.category == 'circle' or self.category == 'semi-circle left' or self.category == 'semi-circle right'
              or self.category == 'ellipse'):
            # Subbing parameterisation x = x_init + v0*t, y = y_init + v1*t into the equation for the circle, 
            # (x - x_c)^2 + (y-y_c)^2 = r^2, gives a quadratic equation in t, At^2 + Bt + C = 0, 
            # with A, B and C as follows:
            
            if self.category != 'ellipse':
                # i.e. we solve using the equation of a circle.
                # Subbing parameterisation x = x_init + v0*t, y = y_init + v1*t into the equation for the circle, 
                # (x - x_c)^2 + (y-y_c)^2 = r^2, gives a quadratic equation in t, At^2 + Bt + C = 0, 
                # with A, B and C as follows:
                A = v0**2 + v1**2
                B = 2*v0*(x_init - self.centre[0]) + 2*v1*(y_init - self.centre[1])
                C = (x_init - self.centre[0])**2 + (y_init - self.centre[1])**2 - self.radius**2
            else:
                # if we're dealing with an ellipse, # need to solve equation for ellipse 
                # (x-x_c)^2/a^2 + (y-y_c)^2/b^2 = 1, using same parameterisation x = x_init + v0*t,
                # y = y_init + v1*t gives a quadratic equation in t, At^2 + Bt + C = 0, with A, B
                # and C given as follows:
                A = (self.b**2) * (v0**2) + (self.a**2) * (v1**2)
                B = (2*v0*(x_init - self.centre[0])*self.b**2) + (2*v1*(y_init - self.centre[1])*self.a**2)
                C = ((x_init - self.centre[0])**2) * (self.b**2) + ((y_init - self.centre[1])**2) * (self.a**2) - (self.a**2)*(self.b**2)
            
            
        
            if B**2 - 4*A*C >= 0:
                # in general, we get two solutions for t: 
                t1 = (-B - np.sqrt(B**2 - 4*A*C))/(2*A)
                t2 = (-B + np.sqrt(B**2 - 4*A*C))/(2*A)

                # it will be up to the user to decide whether self.inner is True or False. True corresponds to 
                # the first possible value of the parameter being realised. The parameter self.inner is used 
                # because it allows us to logically choose whether this will happen based on the situation,
                # rather than writing many unneccessary lines of code.
                if self.inner:
                    t = t1
                else:
                    t = t2

            else:
                # if the discriminant is negative, the ball never hits the boundary:
                t = np.inf
        
        # extra condition to ensure we never hit the wrong side of a semi-circle:
        if t != np.inf and (self.category == 'semi-circle left' or self.category == 'semi-circle right'):
            if x_init + v0*t < self.xlims[0] or x_init + v0*t > self.xlims[1]:
                # i.e. if we're outside limits. corresponds to hitting the other side of the semi circle
                t = np.inf
            
            
        # if collision happens backwards in time, we ignore it.
        # (we set time to 0 after each collision)
        if t <= 0:
            t = np.inf
            
        # storing this within the class to be used later
        self.t = t
            
        return t   # returning for convenience of code to generate collisions in Table class.
    
    def find_boundary_point(self, x_init, y_init, v0, v1):
        '''
        Method to generate the point at which the collision happens. We put this as a seperate method, 
        as we will only call it for values of t for which collisions take place (t will be finite and 
        positive in this case). This will become clear in the generate_collisions() class method of
        the Table class.
        
        Parameters:
            x_init : number
            y_init : number
                x_init and y_init should be the x and y coordinates, respectively, of a previous point 
                on the line segment for which we're trying to find the next collision. When we code the Table class,
                (x_init, y_init) will either be taken as our initial point or the last point where the ball hit
                the boundary.
            
            v0 : number
            v1 : number
                These are the x and y componenets (respectively) of the normalised direction vector before collision.
        '''
        if self.category == 'vertical':
            x_boundary = self.xlims[0]     # need to set this explicitly so we don't run into rounding errors. 
            # if we don't set this explicitly, and set x_boundary = x_init + v0 * self.t, we may encounter a 
            # rounding error where x_boundary is set to be just outside the x limit, rather than being exactly
            # equal to it. This can lead to infinity errors, effectively stopping the rest of the trajectory from
            # generated.
            y_boundary = y_init + v1 * self.t
        elif self.category == 'horizontal':
            x_boundary = x_init + v0 * self.t
            y_boundary = self.ylims[0]     # need to set this explicitly so we don't run into rounding errors. 
            # if we don't set this explicitly, and set y_boundary = y_init + v1 * self.t, we may encounter a 
            # rounding error where y_boundary is set to be just outside the y limit, rather than being exactly
            # equal to it. This can lead to infinity errors, effectively stopping the rest of the trajectory from
            # generated.
        else: 
        # i.e. we're dealing with a curved boundary, which don't have x or y constrained to be some constant.
            x_boundary = x_init + v0 * self.t
            y_boundary = y_init + v1 * self.t
        
        return x_boundary, y_boundary
            
                
    def collide(self, direction_vector, x_boundary=None, y_boundary=None):
        '''
        Method for finding the effects of collisions. This will take in the direction vector and change its components 
        according to the type of boundary, and perhaps where on the boundary the ball collides, as explained
        in detail above.
        
        Parameters: 
            direction_vector : list containing two numbers
                This is the initial direction vector before collision, whose components we will 
                change according to the nature of the collision.
                
            x_boundary (optional) : number
                x coordinate of the boundary point, generated by the find_boundary_point() method
                Defualted to None because we don't require it as an input if the category of our
                boundary is horizontal or vertical.
            
            y_boundary (optional) : number
                y coordinate of the boundary point, generated by the find_boundary_point() method.
                Defaulted to None because we don't require it as an input if the category of our 
                boundary is horizontal or vertical.
        '''
        if self.category == 'horizontal':
            direction_vector[1] *= -1       # y component of the velocity vector is multiplied by -1 in this case
        elif self.category == 'vertical':
            direction_vector[0] *= -1       # x component of the velocity vector is multiplied by -1 in this case
        
        # for any other category, we step back in time and use the reflection transformation matrix, as 
        # explained above:
        elif(self.category == 'circle' or self.category == 'ellipse' 
             or self.category == 'semi-circle left' or self.category == 'semi-circle right'):
            # stepping back in time to just before we hit boundary:
            px = x_boundary - small_step * direction_vector[0]
            py = y_boundary - small_step * direction_vector[1]
            # (px, py) is the coordinate of the point just before we hit the boundary.
            
            # the following should almost never happen, but the code is included in case 
            # the trajectory is initialised in such a way that it occurs. The code is included
            # to avoid errors involving division by zero in this case.
            if x_boundary - self.centre[0] == 0:
                # i.e. if we have to divide by 0 in the next step
                direction_vector[1] *= -1
                return
            # NOT SURE IF THIS IS CORRECT. SHOULD TEST INITIALISING DIRECTION VECTOR IN THIS WAY TO MAKE SURE.
            
            # finding gradient and y-intercept of normal:
            if self.category == 'circle' or self.category == 'semi-circle left' or self.category == 'semi-circle right':
                # simply using y-y_1 = m(x-x_1) with the fact that the normal passes through the boundary and the centre:
                m = (y_boundary - self.centre[1])/(x_boundary - self.centre[0])
                c = -m*x_boundary + y_boundary        
            elif self.category == 'ellipse':
                # elliptical boundary parameterised by x - x_c = acos(p), y - y_c =bsin(p)
                # so that normal is given by a(x-x_c)/cos(p) - b(y-y_c)/sin(p) = a^2 - b^2
                # we also use y - y1 = m(x - x1) to find c:
                
                cos_p = (x_boundary - self.centre[0]) / self.a
                sin_p = (y_boundary - self.centre[1]) / self.b
                
                # the following results are obtained for ellipses not necessarily centred at the origin:
                m = (self.a*sin_p)/(self.b*cos_p)
                c = -m*x_boundary + y_boundary
                        
            # using transformation matrix for reflection in the line y = mx + c (as explained above)
            nx = ((1-m**2)*px)/(1+m**2) + (2*m*py)/(1+m**2) + (-2*m*c)/(1+m**2)
            ny = (2*m*px)/(1+m**2) + ((m**2 - 1)*py)/(1+m**2) + (2*c)/(1+m**2)
            # nx and ny are x and y coordinates of points just after collision
            
            # finding components of directoin vector after collision using this new point:
            direction_vector[0] = nx - x_boundary
            direction_vector[1] = ny - y_boundary
            # we will normalise the new direction vector within the Table class.


### Table class

The majority of the code needed to actually generate trajectories is already written within the Boundary class. In the Table class, the methods above will be combined to generate collisions. Functionality for plotting trajectories, and storing data will also be provided. This data will be used for animation, and analysis of trajectories.

In [ ]:
class Table:
    
    def __init__(self, initial_point, initial_direction_vector, boundaries):
        '''
        Initialisation function for the Table class. Every conceivable trajectory can be characterised solely by the 
        initial point, initial direction vector (i.e. initial velocities) and the set of boundaries within which
        the trajectory is constrained. 
        
        
        Unless the user is generating animations or recurrence plots, they should set ball_speed large enough 
        (significantly larger than the scale of the table) so that intermediate points (points between points of 
        intersection with the boundary) are not stored, as this data is unneccesary. Although the initialisation 
        method (when combined with the generate_collisions() method) does produce some unneccessary data 
        (for example, x_array can be found from intersection_points), the approach taken is to store this data anyway, 
        for convenience of analysis later. The space used up by storing the neccesary data (intersection_points) scales 
        linearly with the number of collisions, whilst the space used up storing this unneccesary data also scales linearly 
        with the number of collisions. Hence, storing the extra data does not drastically effect the optimality of the 
        algorithm for a large number of collisions. 
        
        Parameters:
            initial_point : tuple containing two numbers
                This will be the starting point of the trajectory. To form a trajectory that is of any interest,
                the initial point should be within the set of closed boundaries.
                
            initial_direction_vector : list containing two numbers
                In general, this doesn't have to be normalised - we will normalise in initialisation function.
                Will correspond to the initial componenets of the velocity of the ball. 
                
            boundaries : list
                List should contain objects belonging to the Boundary class.
                Generally, we will input a closed set of boundaries, though it is possible to imagine
                trajectories formed within a set of boundaries that aren't closed (as they can still be constrained
                not to exit the boundary and move off to infinity in some cases)
        '''
        # ensure that direction vector is normalised. We will also normalise each time the direction vector changes.
        # this is important to keep animation points evenly spaced throughout.
        self.direction_vector = normalise(initial_direction_vector)
        
        # initialising boundary set:
        self.boundaries = boundaries
        
        # for collecting data:
        self.intersection_points = [initial_point]     # note that the first element is not an intersection point but the rest will be
        # we don't really use self.intersection_points, but it is kept as a class attribute in case the user wishes to access the data. 
        # self.x_array and self.y_array will provide the same data, just with repetitions
        
        self.anim_points = [initial_point]    # anim_points will be used for the generation of recurrence plots, as well as for animation.
        self.extended_theta_array = [find_theta(initial_direction_vector)]    # should be same length as anim_points. 
        # Although self.extended_theta_array is not used in the project, it is provided in case the user wants to carry out 
        # further analysis. An example of how this can be done in the context of recurrence plots is given at the end of the project.
        self.theta_array = [find_theta(initial_direction_vector)]     # should be same length as x_array, y_array (corresponds to intersection points)
        
        # we also store x and y arrays for convenience of plotting phase spaces later.
        # these correspond exactly to the points in intersection_points, but have (almost) double 
        # the number of elements. This is because theta changes instantaneously at the intersection
        # points, and we want to store each x and y that correspond to a certain theta
        self.x_array = [initial_point[0]]
        self.y_array = [initial_point[1]]
        
        # storing initial data for clear() method:
        self.initial_direction_vector = initial_direction_vector
        self.initial_point = initial_point
        
    def generate_collisions(self, number_of_collisions=10):
        '''
        Method for generating trajectories. The approach we use to generate collisions is explained above in detail.
        
        Parameters:
            number_of_collisions (optional) : positive integer
                (defaulted to 10 collisions)
                Corresponds to the number of times the ball collides with the boundaries before we stop 
                generating more of the trajectory.
                
        Returns:
            self : Table
                This is simply for convenience of code in the future. It allows us to call class methods (or
                find class attributes) in the same line as generating collisions. So we can write something like:
                table.generate_collisions(10).generate_collisions(20).plot_intersections(fig)
        '''
        for i in range(number_of_collisions):
            # renaming variables to make code more readable:
            x_init, y_init = self.intersection_points[-1][0], self.intersection_points[-1][1]
            v0, v1 = self.direction_vector[0], self.direction_vector[1]
                        
            self.boundaries.sort(key=lambda boundary: boundary.find_t(x_init, y_init, v0, v1)) # sorting boundaries in terms of t's
            t = self.boundaries[0].t    # this is the t we will use
            
            # this shouldn't happen for valid trajectories, but provided so that the user can understand what's happening
            # if this error occurs, and not to get stuck in an infinite loop when generating animation points.
            if t == np.inf:
                print("INFINITY ERROR")
                break
            
            theta = find_theta(self.direction_vector)
            
            # data for animation / recurrence plot:
            # these will be intermediate points (between points where the ball hits the boundary)
            t_anim = ball_speed
            while t_anim < t:
                self.anim_points.append((x_init + t_anim*v0, y_init + t_anim*v1))
                self.extended_theta_array.append(theta)
                t_anim += ball_speed
            
            # finding boundary points:
            x_boundary, y_boundary = self.boundaries[0].find_boundary_point(x_init, y_init, v0, v1)
            
            # keeping track of all our data, including data for phase space and recurrence plots.
            self.intersection_points.append((x_boundary, y_boundary))
            self.anim_points.append((x_boundary, y_boundary))
            self.extended_theta_array.append(theta)
            self.x_array.append(x_boundary)
            self.y_array.append(y_boundary)
            self.theta_array.append(theta)
            
            # collision with boundary that is hit first:
            self.boundaries[0].collide(self.direction_vector, x_boundary, y_boundary)
            # we do this towards the end of the for loop so that the necessary data is stored before collision.
            
            # coding for corner collisions:
            if len(self.boundaries) > 1:
                if self.boundaries[1].t - t <= 1e-8:        # i.e. if the ball hits two boundaries at two points in time that are extremely near to each other
                # (in an analytic sense, the 1e-8 could be set to 0, but it is set to some finite number to account for possible rounding errors)    
                    
                    # here we only consider concave (inwards facing) corners between straight (horizontal or vertical) boundaries
                    # could generalise this in theory, but if we just use this it allows us to take care of corners in rectangular billiards.
                    if ((self.boundaries[0].category == 'horizontal' and self.boundaries[1].category == 'vertical')
                        or self.boundaries[0].category == 'vertical' and self.boundaries[1].category == 'horizontal'):
                        # i.e. if we're at or very close to a corner
                        self.boundaries[1].collide(self.direction_vector)
            # if I had more time I'd generalise the above to deal with any sharp edge (non-differentiable) part
            # of the boundary
                    
            # keeping track of data for phase plots:
            # need to do this at end of loop too, as theta has changed within loop.
            self.x_array.append(x_boundary)
            self.y_array.append(y_boundary)
            self.theta_array.append(find_theta(self.direction_vector))
            
            normalise(self.direction_vector)      # normalise to ensure that anim_points are evenly spaced throughout the trajectory.
            
        return self      # for convenice of code later
        
    def clear(self):
        '''
        Method for clearing the trajectory in order to start again if the user wishes to do so.
        Note that if the user wishes to change the initial direction vector after clearing, 
        they can change the class attribute table.initial_direction_vector but they must also change 
        table.direction_vector, table.theta_array and table.extended_theta_array to their corresponding values
        for consistency. If they wish to change the initial point, they must also change the other class 
        attributes determined by initial_point. Since so much needs to be change, it is recommended to simply 
        initialise a new table with new initial conditions and the same boundaries rather than changing all these 
        class attributes if the user wishes to start with different initial conditions. 
        
        Returns:
            self : Table
                This is simply for convenience of code in the future. It allows us to call class methods (or
                find class attributes) in the same line as clearing the table. So we can write something like:
                table.clear().generate_collisions()
        
        '''
        # simply resetting class attributes to what they were initialised as:
        self.direction_vector = normalise(self.initial_direction_vector)
        self.intersection_points = [self.initial_point]     
        self.anim_points = [self.initial_point]
        self.extended_theta_array = [find_theta(self.initial_direction_vector)]
        self.theta_array = [find_theta(self.initial_direction_vector)]
        self.x_array = [self.initial_point[0]]
        self.y_array = [self.initial_point[1]]
        
        return self     # for convenience of code later
        
        
    def plot_intersections(self, fig, colour='blue'):
        '''
        Method to plot trajectories according to their intersection points with the boundaries.
        We also plot the boundaries within this method, using the already defined method within
        the Boundary class.
        
        Parameters:
            fig : matplotlib Figure
                The figure on which we will plot trajectories (and boundaries) 
            colour (optional) : string
                The colour the trajectory will be plotted in (defaulted to blue).
                Provided as an optional parameter as it may be useful to change the colour if we 
                want to plot multiple trajectories on the same figure.
        '''
        # plotting intersection data: 
        plt.plot(self.x_array, self.y_array, color=colour, linewidth=0.5, figure=fig)  
        
        for boundary in self.boundaries:
            boundary.plot(fig)
            
    def get_anim_data(self):
        '''
        Simple method for generating data to be used for animations.
        
        Note that we have to write the actual code for the animation outside the function, explicitly
        in a cell, in order to get it to display. 
        '''
        # we require the 'transpose' of the data for plotting points
        anim_data = np.array(
            [[point[0] for point in self.anim_points], 
             [point[1] for point in self.anim_points]]
        )
            
        return anim_data
        

## Square / rectangular billiards

To start with, consider the familiar game of rectangular billiards. In generating the trajectories, at least with integer boundaries, it becomes clear very quickly that the plots are periodic (and hence regular). 

In [ ]:
# setting figure:
fig1 = plt.figure(figsize=(7, 7/2))
fig1.suptitle('Rectangular billiards table animation')
plt.xlim(-2.1, 2.1)
plt.ylim(-2.1/2, 2.1/2)

# defining boundaries:
rectangular_boundaries = [
    Boundary('horizontal', xlims=[-2, 2], ylims=[-1]),
    Boundary('horizontal', xlims=[-2, 2], ylims=[1]),
    Boundary('vertical', xlims=[-2], ylims=[-1, 1]),
    Boundary('vertical', xlims=[2], ylims=[-1, 1])
]

# setting ball_speed (for animation) and initialising table:
ball_speed = 0.05
rectangular_table = Table(initial_point=(0, 0.73), initial_direction_vector=[1, 0.8], 
                          boundaries=rectangular_boundaries) 
rectangular_table.generate_collisions(30)

# plotting boundaries
for boundary in rectangular_boundaries:
    boundary.plot(fig1)

anim_data = rectangular_table.get_anim_data()

# code for animation:
l, = plt.plot([], [], color='blue')
line_animation = animation.FuncAnimation(fig1, update_line, frames=len(
    anim_data[0])+1, fargs=(anim_data, l), interval=1, blit=True, repeat=False)

plt.grid()
plt.show()

ball_speed = 100       # setting ball_speed back to a large number in order not to generate unneccesary points in the coming calculations

# this system defined has a period of 26 collisions.

These systems of rectangular billiards are closesly related to the mathematical problem of arithmetic billiards. If the ball starts at any point within the rectangle with integer coordinates (where the boundaries are assumed to start and end at integer coordinates), then there are periodic paths unless the recangle sides are coprime. The length of a periodic path (in terms of distance travelled) is $ 2\sqrt{2}lcm(a, b) $, where lcm stands for lowest common multiple$^3$. This idea of integer boundaries can be appropriately scaled to generalise for rational numbers (the ball is a point particle, so scaling in this way is allowed).

The trajectory can also be understood by looking at the phase space formed, as well as the two-dimensional projections of the phase space.

In [ ]:
# 3-D phase space

fig2 = plt.figure()
ax2 = fig2.add_subplot(111, projection='3d')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('theta')
ax2.set_title('3D Phase Space for rectangular billiards')
ax2.plot(rectangular_table.x_array, rectangular_table.y_array, rectangular_table.theta_array, linewidth=1)
plt.show()

In [ ]:
# Plotting x against theta

plt.figure()
plt.plot(rectangular_table.x_array, rectangular_table.theta_array, linewidth='1')
plt.title('Rectangular billiards - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# Plotting y against theta

plt.figure()
plt.plot(rectangular_table.y_array, rectangular_table.theta_array, linewidth='1')
plt.title('Rectangular billiards - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

Since the trajectory is periodic, the ball reaches a point in phase space which it has already visited, and the trajectory continues on the path from this point that it has already completed.

## Circular billiards

The game of mathematical billiards becomes significantly more interesting when boundaries are allowed to be curved as well as straight. First consider the example of a ball travelling in a circular table. The geometry of a circle means that if a ball starts on a line passing through the centre, it will continue being reflected back and forth along this line. Otherwise, the ball never passes through the centre.

In [ ]:
# can generate simple periodic trajectories, such as the following:

# setting figure:
fig3 = plt.figure(figsize=(7, 7))
fig3.suptitle("Circular billiards plot")
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)

# defining boundary:
unit_circle_boundary = Boundary('circle', xlims=None, ylims=None, centre=(0, 0), radius=1)  

ball_speed = 100      # set to this large size if we don't want to generate any intermediate points (generating intermediate points may be useful for animation or recurrence plots)

# defining table, generating and plotting intersections:
circular_table = Table(initial_point=(0, 0.5), initial_direction_vector=[1, 0], boundaries=[unit_circle_boundary]) 
circular_table.generate_collisions(4)
circular_table.plot_intersections(fig3)

plt.grid()
plt.show()


The trajectory along a circular table is spread evenly along the circular boundary (for a large number of collisions)$^4$. This can be seen if a trajectory with no obvious periodicity is taken: 

In [ ]:
fig4 = plt.figure(figsize=(7, 7))
fig4.suptitle("Circular billiards plot")
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)

circular_table = Table(initial_point=(0, 0.5), initial_direction_vector=[1, 1], boundaries=[unit_circle_boundary]) 
circular_table.generate_collisions(1000)
circular_table.plot_intersections(fig4)
plt.grid()
plt.show()

There is a visible 'forbidden region' - a circle (sharing its centre with the boundary) which the ball's trajectory never enters. 

Consider also the two-dimensional projections of the phase space:

In [ ]:
# Plotting x against theta

plt.figure(figsize=(7, 7))
plt.plot(circular_table.x_array, circular_table.theta_array, linewidth='0.1')
plt.title('Circular billiards - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# Plotting y against theta

plt.figure(figsize=(7, 7))
plt.plot(circular_table.y_array, circular_table.theta_array, linewidth='0.1')
plt.title('Circular billiards - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

## Elliptical billiards

Ellipses have interesting geometric properties which make the game of billiards on an elliptical pool table quite interesting. One such geometric property is that the angle made between the lines connecting each focus with a point on the ellipse is the same.

If the initial line that the ball is fired on passes between the foci of the ellipse, all reflected segments will pass between the foci. If a large number of reflections is considered, a new curve appears to form, known as the evolute of the reflected segments (visible below in a darker blue). This curve is in fact a hyperbola which shares its foci with the original ellipse$^5$.

The foci of the ellipse are marked in green. 

In [ ]:
fig5 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig5.suptitle('Elliptical billiards travelling within foci plot')

elliptical_boundary = Boundary('ellipse', xlims=None, ylims=None, centre=(0, 0), radius=None, a=1, b=0.5)
elliptical_table = Table(initial_point=(0, 0.2), initial_direction_vector=[1, 0.9], boundaries=[elliptical_boundary])
elliptical_table.generate_collisions(200).plot_intersections(fig5)

plt.grid()
plt.show()

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(elliptical_table.x_array, elliptical_table.theta_array, linewidth='0.5')
plt.title('Elliptical billiards travelling within foci - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(elliptical_table.y_array, elliptical_table.theta_array, linewidth='0.5')
plt.title('Elliptical billiards travelling within foci - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

The special case of a ball which is initialised to pass through one of the foci is illustrated in the next animation. The geometrical property mentioned above means that after each collision, the ball passes through a focus:

In [ ]:
fig6 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1*elliptical_boundary.a, 1.1*elliptical_boundary.a)
plt.ylim(-1.1*elliptical_boundary.a, 1.1*elliptical_boundary.a)
fig6.suptitle('Animation to show elliptical billiards passing through foci')

elliptical_table_1 = Table(initial_point=(elliptical_boundary.c, 0), initial_direction_vector=[-1, 0.2], boundaries=[elliptical_boundary])
ball_speed = 0.1    # to generate some intermediate points
elliptical_table_1.generate_collisions(500)

elliptical_boundary.plot(fig6)

anim_data = elliptical_table_1.get_anim_data()
l, = plt.plot([], [], color='blue', linewidth=0.5)
line_animation = animation.FuncAnimation(fig6, update_line, frames=len(
    anim_data[0])+1, fargs=(anim_data, l), interval=1, blit=True, repeat=False)

ball_speed = 100    # setting ball_speed back to a large number so we don't generate unneccesary intermediate points in the coming calculations

plt.grid()
plt.show()

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(elliptical_table_1.x_array, elliptical_table_1.theta_array, linewidth='0.2')
plt.title('Elliptical billiards passing through foci - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(elliptical_table_1.y_array, elliptical_table_1.theta_array, linewidth='0.2')
plt.title('Elliptical billiards passing through foci - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

If the original path does not pass between the foci of the ellipse, the entire trajectory will remain outside the foci. If a large number of collisions is considered, a new ellipse appears to form (the evolute of the reflected segments). It can be shown that this ellipse shares its foci with the original ellipse. 

In [ ]:
# try starting outside focus:

fig7 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1*elliptical_boundary.a, 1.1*elliptical_boundary.a)
plt.ylim(-1.1*elliptical_boundary.a, 1.1*elliptical_boundary.a)
fig7.suptitle('Elliptical billiards travelling outside foci')

ball_speed = 100      # resetting to a large ball speed so we don't generate unneccesary data

elliptical_table_2 = Table(initial_point=(0.9, 0), initial_direction_vector=[1, 0.9], boundaries=[elliptical_boundary])
elliptical_table_2.generate_collisions(100).plot_intersections(fig7)

plt.grid()
plt.show()

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(elliptical_table_2.x_array, elliptical_table_2.theta_array, linewidth='0.8')
plt.title('Elliptical billiards travelling outside foci - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(elliptical_table_2.y_array, elliptical_table_2.theta_array, linewidth='0.8')
plt.title('Elliptical billiards travelling outside foci - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

## Bunimovich stadium

The Bunimovich Stadium is a table formed by attaching two semi-circular caps to a rectangular section.

The motion of a billiard in this stadium is ergodic. In other words, for a billiard initialised with random position and velocity, its position becomes uniformly distributed over the whole stadium over time $^6$. This effect is visible in the plot generated below.

In [ ]:
fig8 = plt.figure(figsize=(10, 10/3))
plt.xlim(-3.3, 3.3)
plt.ylim(-3.3/3, 3.3/3)
fig8.suptitle('Stadium billiards plot')

# defining boundaries
stadium_boundaries = [
    Boundary('horizontal', xlims=[-2, 2], ylims=[1]),
    Boundary('horizontal', xlims=[-2, 2], ylims=[-1]),
    Boundary('semi-circle left', xlims=[-3, -2], centre=(-2, 0), radius=1),
    Boundary('semi-circle right', xlims=[2, 3], centre=(2, 0), radius=1)
]


stadium_table = Table(initial_point=(0, 0), initial_direction_vector=[1, 0.74], boundaries=stadium_boundaries)
stadium_table.generate_collisions(100).plot_intersections(fig8)
# we get an if we fire it at the intersection of a semicircle and straight edge. But this should almost never
# happen unless explicitly fired there from initial conditions
    

plt.grid()
plt.show()

It is worth noting that there exist specific sets of initial conditions for which motion should theoretically be periodic and regular (e.g. initial point $(0, 0)$, initial direction vector $(1, 1)$ - the number of these initial conditions  that exist is negligible compared to the number of initial conditions that produce chaotic behaviours for this table)$^7$. However, the computer incurs rounding errors which cause these trajectories to diverge from their theoretical periodic trajectories after a small number of collisions (approximately 15). This further reflects the sensitivity to initial conditions that the stadium table is subject to, as well as reflecting the fact that the model being used has a finite level of precision.

The fact that the trajectories are chaotic can also be visualised using thier sensitivity to initial conditions. If the starting angle is changed by a small amount, the trajectories formed become very different after a small number of collisions, as shown below. In the animation below, the trajectories begin to visibly diverge after only approximately 6 collisions with the boundary.

In [ ]:
# can demonstrate chaotic behaviour using sensitivity to initial conditions:
fig9 = plt.figure(figsize=(9, 9/3))
plt.xlim(-3.3, 3.3)
plt.ylim(-3.3/3, 3.3/3)
fig9.suptitle('Stadium animation to show sensitivity to initial conditions')

ball_speed = 0.01        # setting ball speed small to see animation clearly.

# defining staring angles to be very close to one another:
change_in_angle = 1e-4      # the user is free to change this
angle_1 = np.pi/6           # the user is free to change this
angle_2 = angle_1 + change_in_angle

# initial direction vectors are defined in terms of starting angles here. 
stadium_table_1 = Table(initial_point=(0, 0), 
                        initial_direction_vector=[np.cos(angle_1), np.sin(angle_1)], boundaries=stadium_boundaries)
stadium_table_2 = Table(initial_point=(0, 0), 
                        initial_direction_vector=[np.cos(angle_2), np.sin(angle_2)], boundaries=stadium_boundaries)

for boundary in stadium_boundaries:
    boundary.plot(fig9)
    
anim_data_1 = stadium_table_1.generate_collisions(30).get_anim_data()
anim_data_2 = stadium_table_2.generate_collisions(30).get_anim_data()

#stadium_table_1.plot_intersections(fig9)            
#stadium_table_2.plot_intersections(fig9, 'green')


l_1, = plt.plot([], [], color='blue', linewidth=0.5, label='initial trajectory')
l_2, = plt.plot([], [], color='green', linewidth=0.5, label=f'starting angle changed by {change_in_angle} radians')
stadium_chaos_animation = animation.FuncAnimation(fig9, update_two_lines, frames=len(
    anim_data_1[0])+1, fargs=(anim_data_1, anim_data_2, l_1, l_2), interval=1, blit=True, repeat=False)

ball_speed = 100       # setting ball_speed back to a large number so that unneccesary data is not generated in future calculations

plt.legend(loc='upper left')
plt.grid()
plt.show()



It is also worth looking at how a change in initial conditions on the same order affects a regular system, such as trajectories within a circle (for the sake of comparison):

In [ ]:
fig10 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig10.suptitle('Circular table animation to show (lack of) sensitivity to initial conditions')

ball_speed = 0.1        # setting ball speed small to see animation clearly.

# defining staring angles to be very close to one another:
change_in_angle = 1e-4      # the user is free to change this
angle_1 = np.pi/6           # the user is free to change this
angle_2 = angle_1 + change_in_angle

# initial direction vectors are defined in terms of starting angles here. 
circular_table_1 = Table(initial_point=(0, 0.5), 
                        initial_direction_vector=[np.cos(angle_1), np.sin(angle_1)], boundaries=[unit_circle_boundary])
circular_table_2 = Table(initial_point=(0, 0.5), 
                        initial_direction_vector=[np.cos(angle_2), np.sin(angle_2)], boundaries=[unit_circle_boundary])

unit_circle_boundary.plot(fig10)

anim_data_1 = circular_table_1.generate_collisions(50).get_anim_data()
anim_data_2 = circular_table_2.generate_collisions(50).get_anim_data()

#circular_table_1.plot_intersections(fig10)            
#circular_table_2.plot_intersections(fig10, 'green')

l_1, = plt.plot([], [], color='blue', linewidth=0.5, label='initial trajectory')
l_2, = plt.plot([], [], color='green', linewidth=0.5, label=f'starting angle changed by {change_in_angle} radians')
stadium_chaos_animation = animation.FuncAnimation(fig10, update_two_lines, frames=len(
    anim_data_1[0])+1, fargs=(anim_data_1, anim_data_2, l_1, l_2), interval=1, blit=True, repeat=False)

ball_speed = 100       # setting ball_speed back to a large number so that unneccesary data is not generated in future calculations

plt.legend(loc='upper left')
plt.grid()
plt.show()


In the case of the circular table, when the starting angle is changed by the same amount, the trajectories don't visibly diverge over the same number of collisions.

To understand the behaviour of the trajectories of the Bunimovich Stadium, one can compare the phase space generated to those we have already encountered:

In [ ]:
stadium_table.clear().generate_collisions(1500)

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(stadium_table.x_array, stadium_table.theta_array, linewidth='0.1')
plt.title('Stadium billiards - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(stadium_table.y_array, stadium_table.theta_array, linewidth='0.1')
plt.title('Stadium billiards - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

The phase plots above appear significantly less ordered than those seen that correspond to regular trajectories. However, they don't appear to be completely random - there are some areas of phase space in which the ball is more concentrated than others. The beginnings of fractal images can be seen in the phase plots. 

Chaos in billiard tables can come in different forms. For example, rational polygonal billiards (i.e. polygonal tables with rational angles) can produce chaotic trajectories, but these trajectories will be more predictable than stadium trajectories (and others)$^8$. 

Loosely speaking, chaotic behaviour is produced in the above case due to the combination of boundaries with different geometrical properties (as seen already, circular and rectangular billiard tables produce regular trajectories, but their combination produces chaotic trajectories). With this in mind, the project next investigates tables made up of different combinations of different types of boundaries (curved and straight) to examine their chaos or regularity.

## Exploring other tables: circular boundary with straight inner wall

In [ ]:
fig11 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig11.suptitle('Circular boundary with inner straight wall passing through origin')

inner_wall = Boundary('horizontal', xlims=[0.1, 0.7], ylims=[0])         # requires slight modification to boundary class
# as now need to ensure that the intersection lies within the xlims (and ylims for vertical)

# unit circle boundary already defined
circle_wall_boundaries = [
    Boundary('horizontal', xlims=[0.1, 0.8], ylims=[0]),
    unit_circle_boundary
]

circle_wall_table = Table(initial_point=(-0.2, 0.1), initial_direction_vector=[4, -1], boundaries=circle_wall_boundaries)
circle_wall_table.generate_collisions(100).plot_intersections(fig11)

plt.grid()
plt.show()

The table above appears to generate more regular behaviour. There is a 'forbidden' inner circle which the ball never enters, in a similar way to the original circular table. However, the regularity of the system depends on whether the inner wall boundary passes through the centre of the circle (if extended). If the inner wall boundary is moved so that it doesn't pass through the circle's centre, the trajectory appears to become chaotic.

In [ ]:
# if we place line so that it doesn't pass through centre of circle (even if it's continued,
# we get something that looks more chaotic.

fig12 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig12.suptitle('Circular boundary with inner straight wall passing not through origin')

# change inner wall boundary so that it does not pass through the origin:
circle_wall_boundaries = [
    Boundary('horizontal', xlims=[0.1, 0.7], ylims=[0.2]), 
    unit_circle_boundary
]
    
circle_wall_table = Table(initial_point=(-0.2, 0.1), initial_direction_vector=[1, -1], boundaries=circle_wall_boundaries)
circle_wall_table.generate_collisions(100).plot_intersections(fig12)

plt.grid()
plt.show()

In [ ]:
circle_wall_table.clear().generate_collisions(3000)

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(circle_wall_table.x_array, circle_wall_table.theta_array, linewidth='0.05')
plt.title('Circular table with straight inner wall - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(circle_wall_table.y_array, circle_wall_table.theta_array, linewidth='0.05')
plt.title('Circular table with straight inner wall - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

The phase plots look more similar to those produced by the stadium table than to any other table, which suggests chaotic behaviour. The lack of symmetry in the phase plots (compared to the stadium phase plots) reflects the fact that the inner wall boundary is not defined symmetrically with respect to the x and y axes. The motion is visibly spread relatively evenly around phase space, which connotes ergodicity.

## Square within a circle

Even a square within a circle can produce trajectories which appear somewhat regular, when the square and the circle share the same origin. The appearance of four 'forbidden regions' also becomes apparent. 

In [ ]:
fig13 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig13.suptitle('Square within a circle plot')

# defining boundaries. Unit circle boundary already defined.
square_within_circle_boundaries = [
    Boundary('vertical', xlims=[-0.5], ylims=[-0.5, 0.5]),
    Boundary('vertical', xlims=[0.5], ylims=[-0.5, 0.5]),
    Boundary('horizontal', xlims=[-0.5, 0.5], ylims=[-0.5]),
    Boundary('horizontal', xlims=[-0.5, 0.5], ylims=[0.5]),
    unit_circle_boundary
]

# Collisions with the outside corners of the square are not supported in the code, 
# but this should hardly ever happen unless the ball is explicitly fired at the outside corner.

    
square_within_circle_table = Table(initial_point=(0, 0.7), initial_direction_vector=[1, 0.9],
                                   boundaries=square_within_circle_boundaries)
square_within_circle_table.generate_collisions(1000).plot_intersections(fig13)


plt.grid()
plt.show()

The phase plots produced do not resemble those of a chaotic trajectory:

In [ ]:
square_within_circle_table.clear().generate_collisions(5000)

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(square_within_circle_table.x_array, square_within_circle_table.theta_array, linewidth='0.05')
plt.title('Square within circle table - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(square_within_circle_table.y_array, square_within_circle_table.theta_array, linewidth='0.05')
plt.title('Square within circle table - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

## Circle within a square

Commonly known as a form of Sinai's billiards - these tables produce chaotic behaviours even if the circle and the square share the same centre$^9$.

In [ ]:
fig14 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig14.suptitle('Sinai billiards plot')

# defining boundaries. This as an example of when the inner parameter is set to True.
circle_within_square_boundaries = [
    Boundary(category='circle', inner=True, centre=(0, 0), radius=0.5), 
    Boundary(category='vertical', xlims=[-1], ylims=[-1, 1]),
    Boundary(category='vertical', xlims=[1], ylims=[-1, 1]),
    Boundary(category='horizontal', xlims=[-1, 1], ylims=[-1]),
    Boundary(category='horizontal', xlims=[-1, 1], ylims=[1])
]

circle_within_square_table = Table(initial_point=(0, 0.7), initial_direction_vector=[1, 3], 
                                   boundaries=circle_within_square_boundaries)
    
circle_within_square_table.generate_collisions(500).plot_intersections(fig14)

plt.grid()
plt.show()

Extreme sensitivity to initial conditions can also be demonstrated:

In [ ]:
# can demonstrate chaotic behaviour using sensitivity to initial conditions:
fig15 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig15.suptitle('Sinai billiards animation to show sensitivity to initial conditions')

ball_speed = 0.01        # setting ball speed small to see animation clearly.

# defining staring angles to be very close to one another:
change_in_angle = 1e-4      # the user is free to change this
angle_1 = np.pi/6           # the user is free to change this
angle_2 = angle_1 + change_in_angle

# initial direction vectors are defined in terms of starting angles here. 
sinai_table_1 = Table(initial_point=(0, 0.8), 
                        initial_direction_vector=[np.cos(angle_1), np.sin(angle_1)], boundaries=circle_within_square_boundaries)
sinai_table_2 = Table(initial_point=(0, 0.8), 
                        initial_direction_vector=[np.cos(angle_2), np.sin(angle_2)], boundaries=circle_within_square_boundaries)

for boundary in circle_within_square_boundaries:
    boundary.plot(fig15)
    
anim_data_1 = sinai_table_1.generate_collisions(30).get_anim_data()
anim_data_2 = sinai_table_2.generate_collisions(30).get_anim_data()

#sinai_table_1.plot_intersections(fig15)            
#sinai_table_2.plot_intersections(fig15, 'green')

l_1, = plt.plot([], [], color='blue', linewidth=0.5, label='initial trajectory')
l_2, = plt.plot([], [], color='green', linewidth=0.5, label=f'starting angle changed by {change_in_angle} radians')
stadium_chaos_animation = animation.FuncAnimation(fig15, update_two_lines, frames=len(
    anim_data_1[0])+1, fargs=(anim_data_1, anim_data_2, l_1, l_2), interval=1, blit=True, repeat=False)

plt.legend(loc='upper left')
plt.grid()
plt.show()

ball_speed = 100       # setting ball_speed back to a large number so that unneccesary data is not generated in future plots

In [ ]:
# now considering the phase spaces generated:
circle_within_square_table.clear().generate_collisions(5000)

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(circle_within_square_table.x_array, circle_within_square_table.theta_array, linewidth='0.05')
plt.title('Sinai billiards - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(circle_within_square_table.y_array, circle_within_square_table.theta_array, linewidth='0.05')
plt.title('Sinai billiards - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

The phase space formed resembles that of a chaotic system.

## Mushroom billiards

The mushroom billiard table is an example of a table for which a non-negligible number of both chaotic and regular trajectories coexist$^{10}$. This differs from the examples seen so far - the tables so far that have produced chaotic behaviours only have a negligible number of possible regular trajectories. The simplest version of the table is formed by attaching a semi-circular 'cap' to a rectangular 'stem', as below. The motion in the mushroom table is regular when the billiard never leaves the cap, but becomes chaotic and ergodic when its initial conditions allow it to enter the stem$^{11}$. This is demonstrated below. 

In [ ]:
fig16 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig16.suptitle('Regular mushroom billiards plot')

mushroom_boundaries = [
    Boundary(category='vertical', xlims=[-1], ylims=[-0.5, 0.5]),
    Boundary(category='horizontal', xlims=[-1, 0], ylims=[-0.5]),
    Boundary(category='horizontal', xlims=[-1, 0], ylims=[0.5]),
    Boundary(category='vertical', xlims=[0], ylims=[-1, -0.5]),
    Boundary(category='vertical', xlims=[0], ylims=[0.5, 1]),
    Boundary(category='semi-circle right', xlims=[0, 1], ylims=[-1, 1], centre=(0, 0), radius=1)
]

# initial conditions are defined so that the billiard doesn't enter the stem.
mushroom_table = Table(initial_point=(0, -0.75), initial_direction_vector=[1, 1], boundaries=mushroom_boundaries)
mushroom_table.generate_collisions(100).plot_intersections(fig16)

plt.grid()
plt.show()

In [ ]:
# now chaotic mushroom table:

fig17 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig17.suptitle('Chaotic mushroom billiards plot')

# initial conditions are defined so that the billiard does enter the stem.
mushroom_table_2 = Table(initial_point=(-0.75, 0), initial_direction_vector=[1, 1], boundaries=mushroom_boundaries)
mushroom_table_2.generate_collisions(100).plot_intersections(fig17)

plt.grid()
plt.show()

The corresponding phase space for the mushroom billiard that enters the stem visibly resembles a chaotic phase space:

In [ ]:
mushroom_table_2.clear().generate_collisions(3000)

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(mushroom_table_2.x_array, mushroom_table_2.theta_array, linewidth='0.05')
plt.title('Chaotic mushroom billiards - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(mushroom_table_2.y_array, mushroom_table_2.theta_array, linewidth='0.05')
plt.title('Chaotic mushroom billiards - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

An interesting feature of the phase plot for x against theta is the visible difference between the behaviours in the rectangular and semi-circular regions.

Another interesting phenomenon is that if the semi-circular cap is changed to a semi-elliptical cap, the behaviour becomes regular, whether the ball enters the stem or not:

In [ ]:
fig18 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig18.suptitle('Elliptical mushroom billiards plot - ball not allowed to enter stem')

elliptical_mushroom_boundaries=[
    Boundary(category='ellipse', xlims=None, ylims=None, centre=(0, 0), radius=None, a=1, b=0.5),
    Boundary(category='horizontal', xlims=[-1, -0.5], ylims=[0]),
    Boundary(category='horizontal', xlims=[0.5, 1], ylims=[0]),
    Boundary(category='vertical', xlims=[-0.5], ylims=[-0.25, 0]), 
    Boundary(category='vertical', xlims=[0.5], ylims=[-0.25, 0]),
    Boundary(category='horizontal', xlims=[-0.5, 0.5], ylims=[-0.25])
]

# setting initial conditions so that ball is not allowed to enter stem
elliptical_mushroom_table = Table(initial_point=(0.9, 0.1), initial_direction_vector=[0.1, 0.4], boundaries=elliptical_mushroom_boundaries)
elliptical_mushroom_table.generate_collisions(200).plot_intersections(fig18)
    
plt.grid()
plt.show()

# Code does not support firing at a convex corner.

In [ ]:
# now setting initial conditions so that the ball is allowed to enter the stem
fig19 = plt.figure(figsize=(7, 7))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
fig19.suptitle('Elliptical mushroom billiards plot - ball allowed to enter stem')

elliptical_mushroom_table_2 = elliptical_mushroom_table = Table(initial_point=(0.9, 0.1), initial_direction_vector=[1, 0.4], boundaries=elliptical_mushroom_boundaries)
elliptical_mushroom_table_2.generate_collisions(200).plot_intersections(fig19)

In this new elliptical mushroom table, forbidden regions appear on the left and right. The phase plots formed do not resemble those of a chaotic trajectory:

In [ ]:
elliptical_mushroom_table_2.clear().generate_collisions(2000)

In [ ]:
# corresponding phase space - x against theta:

plt.figure(figsize=(7, 7))
plt.plot(elliptical_mushroom_table_2.x_array, elliptical_mushroom_table_2.theta_array, linewidth='0.05')
plt.title('Elliptical mushroom billiards - x against theta')
plt.xlabel('x')
plt.ylabel('theta')
plt.grid()
plt.show()

In [ ]:
# corresponding phase space - y against theta:

plt.figure(figsize=(7, 7))
plt.plot(elliptical_mushroom_table_2.y_array, elliptical_mushroom_table_2.theta_array, linewidth='0.05')
plt.title('Elliptical mushroom billiards - y against theta')
plt.xlabel('y')
plt.ylabel('theta')
plt.grid()
plt.show()

## Tests of effectiveness

The code was tested to be effective by comparing known results from the literature to the trajectories obtained by the code, including the testing of special cases (such as the elliptical billiard passing through its two foci). Effectiveness can also be tested by comparing results with other models $^{12}$. 

The following code provides a template for testing whether a random trajectory looks reasonable, which should give some indication of how effective the code is:

In [ ]:
# THE FOLLOWING CODE IS PROVIDED FOR DEMONSTRATION AND SHOULD NOT BE RUN

# setting figure:
fig = plt.figure(figsize=(7, 7))
fig.suptitle('Random initial conditions table animation')
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 2.1)

# setting ball_speed (for animation) 
ball_speed = 0.05

# setting random initial conditions (if ball starts outside the table, code should produce infinity error, 
# otherwise, code should generate collisions that appear reasonable
point_rand = (2*np.random.rand() - 1, 2*np.random.rand() - 1)      # 2*np.random.rand() - 1 will generate a random number between -1 and 1
direc_rand = [2*np.random.rand() - 1, 2*np.random.rand() - 1] 

table = Table(intital_point=point_rand, initial_direction_vector=direc_rand, boundaries=table_boundaries) 
# table_boundaries should be a list of instances of the Boundary class already defined

table.generate_collisions(50)      # table is an already defined instance of the table class

# plotting boundaries
for boundary in table.boundaries:
    boundary.plot(fig2)

anim_data = table.get_anim_data()

# code for animation:
l, = plt.plot([], [], color='blue')
line_animation = animation.FuncAnimation(fig, update_line, frames=len(
    anim_data[0])+1, fargs=(anim_data, l), interval=1, blit=True, repeat=False)

plt.grid()
plt.show()

ball_speed = 100       # setting ball_speed back to a large number in order not to generate unneccesary points in the coming calculations


## Further investigations

If able to expand the scope of the project, boundaries would be generalised further to include include lines of any gradients, circular and elliptical arcs between any two angles and perhaps other geometries such as parabolic or hyperbolic.

A further analysis of phase space would also be included. Recurrence plots would be investigated, as well as Lyapunov exponenets in order to more rigorously classify chaotic behaviour. An example of how recurrence plots could be coded for using the framework of the project is shown below:

In [ ]:
# THIS CELL IS FOR DEMONSTRATING HOW THE CODE COULD WORK, AND SHOULD NOT BE RUN

'''
Put simply, recurrence plots allow for visualisation how often points in phase space are revisited, 
or nearly revisited. Note that the time taken to run the code below scales with n^2, where n is the 
number of anim_points - can take a very long time for large n, so not necessarily practical and some
approximations may have to be made. Analysis of recurrence plots would allow for a more rigorous structure
to determine whether a billiard's dynamics are chaotic or regular.
'''

ball_speed = 0.01       # setting small ball_speed to generate lots of animation points, so that i and j below scale roughly with time
epsilon = 0.1           # this can also be changed - corresponds to how close the points have to be in order for a black dot to be plotted

fig1 = plt.figure(figsize=(7, 7))
table.clear().generate_collisions(10000)     # generating a large number of collisions on an already defined table (table is an instance of the Table class)

# iterating through points twice. Computing time scales like O(n^2) due to the two nested for loops
for i in range(len(stadium_table.anim_points)):
    for j in range(i, len(stadium_table.anim_points) - 1):
        point_1 = table.anim_points[i]
        point_2 = table.anim_points[j]
        
        x1, y1, theta1 = point_1[0], point_1[1], table.extended_theta_array[i]
        x2, y2, theta2 = point_2[0], point_2[1], table.extended_theta_array[j]
        
        # we consider the euclidean distance between the points in phase space
        distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (theta1 - theta2)**2)
        
        # by definition of the function R(i, j):
        if distance <= epsilon:
            R = 1
        else:
            R = 0
            
        if R == 1:
            plt.plot([i], [j], marker='o', markersize=0.1, color='black', figure=fig1)
            plt.plot([j], [i], marker='o', markersize=0.1, color='black', figure=fig1)

plt.grid()
plt.show()

## References

1: http://www.cns.gatech.edu/~predrag/courses/PHYS-7224-07/billiards.pdf

2: https://plato.stanford.edu/entries/ergodic-hierarchy/

3: Perucca, Antonella (April 24, 2018). "Arithmetic Billiards". Plus Magazine. University of Cambridge.

4: A Survey of Dynamical Billiards, Markus Himmelstrand, Victor Wilén Department of Mathematics, Analysis
Royal Institute of Technology (KTH)

5: https://cage.ugent.be/~hs/billiards/index.html

6: Leonid A. Bunimovich (1979), On the ergodic properties of nowhere dispersing billiards

7: https://blogs.ams.org/visualinsight/2016/11/15/bunimovich-stadium/

8: https://plus.maths.org/content/chaos-billiard-table

9: Ya. G. Sinai, "Dynamical Systems with Elastic Reflections"

10: Bunimovich L. A. (2001), Mushrooms and other billiards with divided phase space

11: Mason A. Porter and Steven Lansel, Mushroom Billiards

12: https://uk.mathworks.com/matlabcentral/fileexchange/10692-billiard-simulator - an example of a model which can be used against the one in this project

Word count (exluding references): 3399 words